# Рекомендация тарифов

<a class='anchor' id='content'></a>
## Содержание

1. [Описание проекта](#desc)
2. [Получение выборок данных](#get)
3. [Ход исследования](#research)
4. [Результаты исследования](#result)
5. [Итоговая проверка модели](#check)

<a class='anchor' id='desc'></a>
## 1. Описание проекта. Исходные данные

 Многие клиенты пользуются архивными тарифами. Необходимо построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных ранее была выполнена.

#### Цель исследования
Построить модель с максимально большим значением `accuracy`.


#### Задачи исследования
* довести долю правильных ответов до 0.75 и выше
* проверить `accuracy` на тестовой выборке




Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:

- `сalls` — количество звонков,
- `minutes` — суммарная длительность звонков в минутах,
- `messages` — количество sms-сообщений,
- `mb_used` — израсходованный интернет-трафик в Мб,
- `is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).



<a class='anchor' id='desc'></a>
### Знакомство с данными


In [ ]:
!pip install jupyter-black -q

In [ ]:
# помощник писать код красиво

import jupyter_black

jupyter_black.load()

# необходимые библиотеки
import pandas as pd
from IPython.display import display
import numpy as np
import warnings

# импортируем структуры данных для классификации и подсчета метрик
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier

# Скроем лишние предупреждения
warnings.filterwarnings("ignore")

# Настройки Pandas для вывода всех столбцов
pd.set_option("display.max_columns", None)

In [ ]:
# чтение датасета
try:
    data = pd.read_csv("datasets/users_behavior.csv", sep=",")
except:
    data = pd.read_csv(
        "https://code.s3.yandex.net/datasets/users_behavior.csv", sep=","
    )

display(data.head())
data.info()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Полученные данные по всем пользователям полны, пропусков нет. Числовые типы в столбцах соответсвуют логике их содержимого, оставим на этом этапе без изменений. Проверим столбец `is_ultra` на встречающиеся значения, так как там не должно быть никаких других значений кроме 0 и 1

In [ ]:
data["is_ultra"].value_counts(normalize=True)

,proportion
is_ultra,
0,0.693528
1,0.306472


**Вывод:** Исходя из вводных данных нам предстоит решить задачу двоичной классификации и определить какой тариф предпочтительнее для пользователей которые еще не перешли на тарифы "Смарт" и "Ультра". Целевым признаком в нашем случае будет столбец `is_ultra`. Значения 1 в нем составляют примерно 30% от всех данных. Это соотношение желательно сохранить при разделении выборок для обучения и тестирования моделей

[назад к содержанию](#content)

<a class='anchor' id='get'></a>
## 2. Получение выборок




До обучения модели разобъем данные на обучающую, валидационную и тестовую выборки. Тестовый набор данных изначально отсутствует,поэтому разделим данные в соотношении 3:1:1, оставив тестовой выборке 20% данных.
Таким образом в итоге у нас будет:
- Обучающая выборка: 60%
- Валидационная выборка: 20%
- Тестовая выборка: 20%



In [ ]:
# обработаем копию датасета чтобы работать с ней, а не с оригинальной таблицей
df = data.copy()

features = df.drop(["is_ultra"], axis=1)
target = df["is_ultra"]

# разобьем на выборки, сначала выделим тестовую, затем валидационную
train_valid, df_test = train_test_split(
    df, test_size=0.2, random_state=777, stratify=target
)
df_train, df_valid = train_test_split(
    train_valid,
    test_size=0.25,
    random_state=777,
)
display(df_train.shape, df_valid.shape, df_test.shape)

(1928, 5)

(643, 5)

(643, 5)

In [ ]:
b# проверим выборки на сбалансированность по признаку

display("Обучающая выборка:")
display(df_train["is_ultra"].value_counts(normalize=True))

display("Валидационная выборка:")
display(df_valid["is_ultra"].value_counts(normalize=True))

display("Тестовая выборка:")
display(df_test["is_ultra"].value_counts(normalize=True))

'Обучающая выборка:'

,proportion
is_ultra,
0,0.68361
1,0.31639


'Валидационная выборка:'

,proportion
is_ultra,
0,0.723173
1,0.276827


'Тестовая выборка:'

,proportion
is_ultra,
0,0.693624
1,0.306376


[назад к содержанию](#content)

<a class='anchor' id='research'></a>
## 3. Исследование модели

- 3.1. [Решающее дерево](#dts)
- 3.2. [Случайный лес](#random_forest)
- 3.3. [Логистическая регрессия](#logistic)




Для выбора наилучшей обучающей модели для определения тарифа изучим несколько моделей, меняя гиперпараметры и на основании исследования выберем наиболее подходящую нам

<a class='anchor' id='dtc'></a>
### 3.1 Модель решающего дерева

In [ ]:
# переменные для признаков и целевого признака обучающей выборки
train_features = df_train.drop(["is_ultra"], axis=1)
train_target = df_train["is_ultra"]
# переменные для признаков и целевого признака валидационной выборки
valid_features = df_valid.drop(["is_ultra"], axis=1)
valid_target = df_valid["is_ultra"]
# создание модели
model_dtc = DecisionTreeClassifier(random_state=777)

# Обучение модели
model_dtc.fit(train_features, train_target)

DecisionTreeClassifier(random_state=777)

In [ ]:
train_predictions = model_dtc.predict(train_features)
accuracy_learn = accuracy_score(train_target, train_predictions)
accuracy_learn

1.0

На обучающей выборке точность предсказаний равна 1, значит модель корректно обучилась и не делает ошибок при определении значений столбца `is_ultra` в обучающей выборке `df_train`

Оценим модели с различным гиперпараметром max_depth который отвечает за максимальную глубину дерева решений
   - Для глубины дерева от 1 до 10  создадим новую модель, обучим её и оценим точность на валидационной выборке.
   - Результаты выведем на экран.

In [ ]:
# Поиск лучших параметров
best_accuracy = 0
best_params = {}

for depth in range(1, 11):
    for criterion in ["gini", "entropy"]:
        model_dtc = DecisionTreeClassifier(
            random_state=777,
            max_depth=depth,
            criterion=criterion,
        )

        model_dtc.fit(train_features, train_target)
        valid_predictions = model_dtc.predict(valid_features)
        accuracy = accuracy_score(valid_target, valid_predictions)

        print(f"max_depth = {depth}, criterion = {criterion} : {accuracy:.4f}")

        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_params = {"max_depth": depth, "criterion": criterion}

print(f"\nЛучшие параметры: {best_params}")
print(f"Лучшая accuracy: {best_accuracy:.4f}")

max_depth = 1, criterion = gini : 0.7667
max_depth = 1, criterion = entropy : 0.7667
max_depth = 2, criterion = gini : 0.7823
max_depth = 2, criterion = entropy : 0.7823
max_depth = 3, criterion = gini : 0.7916
max_depth = 3, criterion = entropy : 0.7916
max_depth = 4, criterion = gini : 0.7978
max_depth = 4, criterion = entropy : 0.7916
max_depth = 5, criterion = gini : 0.7978
max_depth = 5, criterion = entropy : 0.7838
max_depth = 6, criterion = gini : 0.7947
max_depth = 6, criterion = entropy : 0.7994
max_depth = 7, criterion = gini : 0.7978
max_depth = 7, criterion = entropy : 0.8009
max_depth = 8, criterion = gini : 0.7947
max_depth = 8, criterion = entropy : 0.8103
max_depth = 9, criterion = gini : 0.8009
max_depth = 9, criterion = entropy : 0.8103
max_depth = 10, criterion = gini : 0.7869
max_depth = 10, criterion = entropy : 0.7745

Лучшие параметры: {'max_depth': 8, 'criterion': 'entropy'}
Лучшая accuracy: 0.8103


In [ ]:
# Создаем финальную модель с лучшими параметрами
final_model = DecisionTreeClassifier(
    random_state=777,
    **best_params  # распаковываем лучшие параметры
)

final_model.fit(train_features, train_target)

# Тестируем на тестовой выборке
test_predictions = final_model.predict(test_features)
test_accuracy = accuracy_score(test_target, test_predictions)

print(f"Accuracy на тестовой выборке: {test_accuracy:.4f}")

Лучшая accuracy соответвует параметру `max_depth` = 8, критерий = `entropy`

Gini фокусируется на вероятности случайного выбора разных классов, тогда как Entropy оценивает уровень неопределенности в данных. Entropy более чувствительна к изменениям в распределении классов, в то время как Gini может быть менее чувствителен к малым изменениям.

<a class='anchor' id='random_forest'></a>
### 3.2 Модель случайного леса


Для построения модели случайного леса зададим параметры

`n_estimators от 1 до 10:`  количество  деревьев решений, которые будут использованы в модели.

`max_depth от 1 до 5:`   ограничивает максимальную глубину каждого отдельного дерева в модели.

- Установка значения `max_depth` помогает контролировать сложность модели. Более глубокие деревья могут лучше подстраиваться под данные, но также имеют больший риск переобучения, особенно если данные шумные.

In [ ]:
best_model = None
best_result = 0
for est in range(1, 11):
    for depth in range(1, 11):
        model_rfc = RandomForestClassifier(
            random_state=777, max_depth=depth, n_estimators=est
        )
        model_rfc.fit(
            train_features, train_target
        )  # обучение  на тренировочной выборке
        result = model_rfc.score(valid_features, valid_target)
        if result > best_result:
            best_model = model_rfc  # сохраняем наилучшую модель
            best_result = result  # сохраняем наилучшее значение метрики accuracy на валидационных данных

print("Accuracy наилучшей модели на валидационной выборке:", best_result)
print("Наилучшая модель:")
print("n_estimators:", best_model.n_estimators)
print("max_depth:", best_model.max_depth)

Accuracy наилучшей модели на валидационной выборке: 0.8087091757387247
Наилучшая модель:
n_estimators: 7
max_depth: 7


<a class='anchor' id='logistic'></a>
### 3.3 Модель логистической регрессии

Логистическая регрессия — это статистический метод, используемый для бинарной классификации, который моделирует вероятность принадлежности объекта к одному из двух классов.

Настройка LogisticRegression

Обязательные параметры для перебора**
- `C` (регуляризация) → [0.001, 0.01, 0.1, 1, 10, 100]
- `solver` (алгоритм) → ['lbfgs', 'liblinear', 'newton-cg', 'sag', 'saga']
- `max_iter` (итерации) → [500, 1000, 2000]
- `class_weight` → [None, 'balanced'] ⚠️ при дисбалансе классов

** Важные ограничения совместимости**
- `liblinear` → только `l1` и `l2` penalty
- `lbfgs/newton-cg/sag` → только `l2` или `None` penalty  
- `saga` → поддерживает все penalty включая `elasticnet`
- `elasticnet` → требует `l1_ratio`

 **Рекомендации по использованию**
1. **Всегда используй StandardScaler** для solver'ов: sag, saga, lbfgs, newton-cg
2. **При дисбалансе классов** пробуй class_weight='balanced'
3. **Для больших данных** используй solver='sag' или 'saga'
4. **Для L1-регуляризации** только solver='liblinear' или 'saga'

**Интерпретация результатов**
- **Высокое C (1-100)** → слабая регуляризация, риск переобучения
- **Низкое C (0.001-0.1)** → сильная регуляризация, риск недобучения
- **class_weight='balanced'** → улучшает recall для миноритарных классов

**Производительность**
- `n_jobs=-1` → использует все ядра процессора
- `cv=5` → оптимальный баланс точности и скорости
- `verbose=1` → отслеживание прогресса при большом количестве комбинаций

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Базовый пайплайн
pipeline = Pipeline(
    [
        ("scaler", StandardScaler()),
        ("logreg", LogisticRegression(random_state=777, n_jobs=-1)),
    ]
)

# Основные параметры включая class_weight
param_grid = {
    "scaler": [StandardScaler(), None],
    "logreg__solver": ["lbfgs", "liblinear"],
    "logreg__C": [0.01, 0.1, 1, 10, 100],
    "logreg__max_iter": [1000, 2000],
    "logreg__class_weight": [None, "balanced"],  # ← добавлен class_weight
    "logreg__fit_intercept": [True, False],  # ← добавлен fit_intercept
}

grid_search = GridSearchCV(
    pipeline, param_grid, cv=3, scoring="accuracy", n_jobs=-1, verbose=1
)

grid_search.fit(train_features, train_target)

print("Лучшие параметры:", grid_search.best_params_)
print("Лучшая accuracy:", grid_search.best_score_)

Fitting 3 folds for each of 160 candidates, totalling 480 fits
Лучшие параметры: {'logreg__C': 0.1, 'logreg__class_weight': None, 'logreg__fit_intercept': True, 'logreg__max_iter': 1000, 'logreg__solver': 'liblinear', 'scaler': StandardScaler()}
Лучшая accuracy: 0.7427492171463915


In [ ]:
best_model_lr = None
best_result_lr = 0

for solver in ["lbfgs", "liblinear"]:
    for i in [500, 1000, 1500]:
        model_lr = LogisticRegression(random_state=777, solver=solver, max_iter=i)
        model_lr.fit(train_features, train_target)  # обучение на тренировочной выборке
        valid_predictions = model_lr.predict(valid_features)
        accuracy_lr = accuracy_score(valid_target, valid_predictions)
        if accuracy_lr > best_result_lr:
            best_model_lr = model_lr  # сохраняем наилучшую модель
            best_result_lr = accuracy_lr  # сохраняем наилучшее значение метрики accuracy на валидационных данных
print(
    "Accuracy наилучшей модели логистической регрессии на валидационной выборке:",
    best_result_lr,
)
print("Наилучшая модель:")
print("solver:", best_model_lr.solver)
print("max_iter:", best_model_lr.max_iter)

Accuracy наилучшей модели логистической регрессии на валидационной выборке: 0.7636080870917574
Наилучшая модель:
solver: liblinear
max_iter: 500


Более простая модель без применения Grid Search показала лучший результат. Ключевое отличие: GridSearchCV оптимизирует под среднюю производительность на кросс-валидации, а не под конкретную валидационную выборку, как это делает классическая модель линейной регрессии

Вероятнее всего:

GridSearchCV выбрал масштабирование, которое ухудшило результат для твоих данных

Простая модель попала в "удачную" комбинацию параметров для конкретной валидационной выборки



Полученные результаты хуже чем в методах случайного леса и решающего древа. Более низкая точность может быть связана с особенностями модели:
- Неэффективна для сложных зависимостей (например, нелинейные).
- Чувствительна к выбросам.

Оба этих фактора  имеют место в исходных данных

[назад к содержанию](#content)

<a class='anchor' id='result'></a>
## 4. Результаты исследования. Вывод
**Сравнение моделей**

- дерево решений,
- случайный лес,
- логистическая регрессия.

 Оценим модели по качеству (accuracy) и скорости работы.
1. Качество (accuracy). Это самый важный критерий для бизнеса: чем выше качество, тем больше прибыли приносит продукт.
 - Самое высокое качество  **0.81** у решающего дерева с параметрами `max_depth`=9, `criterion="entropy"`
 - На втором месте — у случайного леса с параметрами `n_estimators:` 4 и `max_depth:` 5 .  **0.80**
 - Самое низкое качество предсказания у логистической регрессии.

2. Скорость работы.
Высокая скорость работы у логистической регрессии: у неё меньше всего параметров, однако скорость работы зависит от количества итераций.
Скорость решающего дерева тоже высокая и зависит от глубины. В нашем случае глубина всего 4 признакаЯ, это немного.
Случайный лес медленнее всех: чем больше деревьев, тем неторопливее работает модель. В нашем случае деревьев 4

Учитывая что параметры модели решающего древа относительно невелики, принимаем эту модель со значением **accuracy = 0.81** за наиболее точную и проверим ее на тестовой выборке.

[назад к содержанию](#content)


<a class='anchor' id='check'></a>
## 5. Проверка модели на тестовой выборке


На тестовой выборке проверим выбранную модель решающего дерева

In [ ]:
test_features = df_test.drop(["is_ultra"], axis=1)
test_target = df_test["is_ultra"]

In [ ]:
# 1. Создаем и обучаем финальную модель с лучшими параметрами
final_model = DecisionTreeClassifier(random_state=777, **best_params)
final_model.fit(train_features, train_target)

# получим предсказания для выборок
train_predictions = final_model.predict(train_features)
valid_predictions = final_model.predict(valid_features)
test_predictions = final_model.predict(test_features)

# 3. Accuracy на всех выборках
accuracy_train = accuracy_score(train_target, train_predictions)
accuracy_valid = accuracy_score(valid_target, valid_predictions)
accuracy_test = accuracy_score(test_target, test_predictions)

print(f"Accuracy на ТРЕНИРОВОЧНОЙ выборке: {accuracy_train:.4f}")
print(f"Accuracy на ВАЛИДАЦИОННОЙ выборке: {accuracy_valid:.4f}")
print(f"Accuracy на ТЕСТОВОЙ выборке: {accuracy_test:.4f}")

# 4. Анализ переобучения (сравниваем train vs test)
difference = accuracy_train - accuracy_test
if difference > 0.05:
    print(f"⚠️ Разница {difference:.4f} - возможное переобучение")
elif difference < -0.02:
    print(f"🔍 Разница {difference:.4f} - возможно недобучение")
else:
    print("✅ Модель стабильна")

Accuracy на ТРЕНИРОВОЧНОЙ выборке: 0.8413
Accuracy на ВАЛИДАЦИОННОЙ выборке: 0.8103
Accuracy на ТЕСТОВОЙ выборке: 0.7947
✅ Модель стабильна



**Выводы исследования:**

Модель **решающего дерева** с параметрами `max_depth=8` и `criterion=entropy` демонстрирует:

- **Высокую точность**: 0.8103 на валидационной выборке
- **Хорошую стабильность**: 0.7947 на тестовой выборке
- **Минимальное переобучение**: разница между тренировочной и тестовой accuracy составляет всего 0.0466

**Рекомендация:**
Модель показывает сбалансированные результаты и рекомендуется к внедрению для классификации пользователей тарифов "SMART" и "ULTRA". Стабильность показателей на различных выборках свидетельствует о хорошей обобщающей способности модели.

Одинаковое значение accuracy (точности) на тестовой и тренировочной выборках в модели решающего дерева может говорить о нескольких вещах:

1. **Непереобучение**: Если модель показывает схожие результаты на валидационной и тестовой выборках, это может указывать на то, что она не переобучена.

2. **Сложность задачи**: Соответсвует выбранной нами модели и гиперпараметрам

3. **Данные**: Если обе выборки имеют схожие характеристики и распределение данных, модель может хорошо работать на обеих выборках.

4. **Точность**:о качестве модели нельзя однозначно судить по одному значению accuracy = 0.8. Рекомендуется рассмотреть дополнительные критерии оценки модели

**Итог:**
для принятия решений о переводе пользователей на новые тарифы из линейки "SMART" и "ULTRA" рекомендую применять модель обучения решающего дерева `final_model` этого исследования с параметрами с параметрами `max_depth=8` и `criterion=entropy`

### 5.1. Проверка модели на адекватность

Для проверки на адеватность преположим что другая случайно выбранная модель предсказала назначения тарифов пользователям исходя из значения в столбце `minutes` - тем у кого значение в этом столбце выше среднего она присвоила тариф ULTRA. Определим ее accuracy

In [ ]:
df_copy_1 = df.copy()
mean_minutes = df_copy_1["minutes"].mean()
df_copy_1["is_ultra"] = (df_copy_1["minutes"] > mean_minutes) * 1
df_copy_1.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,1
2,77.0,467.66,86.0,21060.45,1
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [ ]:
accuracy_random_1 = accuracy_score(df["is_ultra"], df_copy_1["is_ultra"])
print(f"Accuracy случайной модели распределния тарифов: {accuracy_random_1:.2f}")

Accuracy случайной модели распределния тарифов: 0.56


<div class="alert alert-info">
    <b>Комментарий студента </b> <br/>
эту проверку на адекватность тоже удалять, она некорректна скорее всего. Преподаватель в пачке посоветовал немного другой путь да, я подумал что импровизация возможна)
    

</div>

In [ ]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score

# cоздаем и обучаем базовую модель
dummy_model = DummyClassifier(strategy="constant", constant=0)
dummy_model.fit(train_features, train_target)

dummy_accuracy = accuracy_score(test_target, dummy_model.predict(test_features))
print(
    f"Accuracy базовой модели, которая предсказывает всем пользователям \n\
  константное значение: {dummy_accuracy:.2f}"
)

Accuracy базовой модели, которая предсказывает всем пользователям 
  константное значение: 0.69


**Вывод:**
  Наша модель  случайного леса случайного леса `final_model` с параметрами `max_depth=8` и `criterion=entropy`  и с `accuracy` 0.79 показывает значительно лучшее качество по сравнению со случайной моделью. Это говорит о том, что эта модель способна делать более точные предсказания, чем случайный выбор.

  [назад к содержанию](#content)